1. Gerekli kütüphaneler

In [9]:
# Gerekli kütüphaneler
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2. Veriyi okuma

In [10]:
# Veri yolu
file_path = "survey.csv"

# Veriyi oku
df = pd.read_csv(file_path)

# İlk birkaç satıra göz at
df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,Male,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


3. Veri kontrolü

In [11]:
# Sütun bilgisi
print("🔢 Sütun Sayısı:", df.shape[1])
print("\n📋 Eksik Veri Dağılımı:")
print(df.isnull().sum().sort_values(ascending=False))

# Yaş sınırlarını da kontrol edelim
print("\n🎂 Yaş Aralığı:", df['Age'].min(), "–", df['Age'].max())

🔢 Sütun Sayısı: 27

📋 Eksik Veri Dağılımı:
comments                     1095
state                         515
work_interfere                264
self_employed                  18
seek_help                       0
obs_consequence                 0
mental_vs_physical              0
phys_health_interview           0
mental_health_interview         0
supervisor                      0
coworkers                       0
phys_health_consequence         0
mental_health_consequence       0
leave                           0
anonymity                       0
Timestamp                       0
wellness_program                0
Age                             0
benefits                        0
tech_company                    0
remote_work                     0
no_employees                    0
treatment                       0
family_history                  0
Country                         0
Gender                          0
care_options                    0
dtype: int64

🎂 Yaş Aralığı: -1726 – 99

4. Veri temizliği

In [12]:
# 1️⃣ Gereksiz sütunları sil
df.drop(columns=["Timestamp", "comments", "state"], inplace=True)

# 2️⃣ Geçersiz yaşları filtrele
df = df[(df["Age"] >= 15) & (df["Age"] <= 80)]

# 3️⃣ Son kontrol
print("🧹 Yeni veri şekli:", df.shape)

🧹 Yeni veri şekli: (1251, 24)


5. Eksik veri doldurma

In [13]:
# 1️⃣ 'self_employed' sütunu (çoğunlukla 'No' olduğu varsayımı)
df["self_employed"].fillna("No", inplace=True)

# 2️⃣ 'work_interfere' sütunu için eksik değerleri 'Don’t know' ile doldur
df["work_interfere"].fillna("Don’t know", inplace=True)

# Kontrol
print("✅ Kalan eksik değer sayısı:", df.isnull().sum().sum())

✅ Kalan eksik değer sayısı: 0


/var/folders/dm/hckhrzb5207cp2xy2c6ts38c0000gn/T/ipykernel_19644/988773537.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["self_employed"].fillna("No", inplace=True)
/var/folders/dm/hckhrzb5207cp2xy2c6ts38c0000gn/T/ipykernel_19644/988773537.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

6. Encoding

In [14]:
# Hedef değişkeni ayır
y = df["treatment"]

# Geri kalanları özellik olarak al
X = df.drop("treatment", axis=1)

# Tüm kategorik sütunları LabelEncoder ile encode et
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])

# Hedef değişkeni de encode et
y = LabelEncoder().fit_transform(y)

# Kontrol
print("✅ X şekli:", X.shape)
print("✅ y sınıfları:", np.unique(y))

✅ X şekli: (1251, 23)
✅ y sınıfları: [0 1]


7. Train ve test olarak ayırma

In [15]:
# 1️⃣ Eğitim/Test ayrımı
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 2️⃣ StandardScaler ile ölçekle
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Kontrol
print("✅ Eğitim şekli:", X_train_scaled.shape)
print("✅ Test şekli:", X_test_scaled.shape)

✅ Eğitim şekli: (1000, 23)
✅ Test şekli: (251, 23)


8. Ön işlenmiş setlerini kaydetme

In [16]:
# Pandas DataFrame'e geri dönüştür
train_df = pd.DataFrame(X_train_scaled, columns=X.columns)
train_df["treatment"] = y_train

test_df = pd.DataFrame(X_test_scaled, columns=X.columns)
test_df["treatment"] = y_test

# CSV olarak kaydet
train_df.to_csv("normal/train_mental.csv", index=False)
test_df.to_csv("normal/test_mental.csv", index=False)

print("✅ Veriler başarıyla kaydedildi.")

✅ Veriler başarıyla kaydedildi.
